In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from environment import *

from setting import *

In [ ]:
path_dict = path(
    TITLE,
    ELEMENTS,
    NMF_K,
    WT_HCC_K,
    H_HCC_K,
    UPLOAD_TO_PLOTLY,
)

In [ ]:
selected_signal_matrix = pd.read_table(
    path_dict['selected_signal_matrix_file_path'],
    index_col=0,
).fillna(0)

In [ ]:
k_nmfcc = ccal.nmf_consensus_cluster_with_multiple_k(
    selected_signal_matrix,
    NMF_KS,
    n_job=MAX_N_JOB,
    plot_df=False,
    plot_w=selected_signal_matrix.shape[0] < 3200,
    plot_h=selected_signal_matrix.shape[1] < 3200,
    directory_path=path_dict['nmf_directory_path'],
)

In [ ]:
w = k_nmfcc['K{}'.format(NMF_K)]['w']

h = k_nmfcc['K{}'.format(NMF_K)]['h']

for wt_or_h, wt_or_h_matrix in (
    (
        'wt',
        w.T,
    ),
    (
        'h',
        h,
    ),
):
    
    title = 'NMF K={} {}'.format(
        NMF_K,
        wt_or_h,
    )
    
    if wt_or_h_matrix.shape[1] < 3200:
        
        ccal.plot_heat_map(
            wt_or_h_matrix,
            normalization_axis=0,
            normalization_method='-0-',
            cluster_axis=1,
            title=title,
            xaxis_title=wt_or_h_matrix.columns.name,
            yaxis_title=wt_or_h_matrix.index.name,
        )
        
    ccal.make_comparison_panel(
        wt_or_h_matrix,
        wt_or_h_matrix,
        title=title,
        name_0=wt_or_h_matrix.index.name,
        name_1=wt_or_h_matrix.index.name,
    )

In [ ]:
for wt_or_h, target_x_sample, features in (
    (
        'wt',
        w.T,
        selected_signal_matrix.T,
    ),
    (
        'h',
        h,
        selected_signal_matrix,
    ),
):
    
    ccal.make_match_panels(
        target_x_sample,
        {
            'Selected Signal': {
                'df': features,
                'data_type': 'continuous',
                'emphasis': 'high',
            },
            'Input (Feature-by-Sample)': {
                'df': pd.read_table(
                    FEATURE_X_SAMPLE_FILE_PATH,
                    index_col=0,
                ),
                'data_type': 'continuous',
                'emphasis': 'high',
            },
        },
        n_job=MAX_N_JOB,
        n_required_for_match_function=None,
        extreme_feature_threshold=EXTREME_FEATURE_THRESHOLD,
        plot_features_std_max=None,
    )